In [1]:
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
raw_data = pd.read_excel('data/raw_data.xlsx')

### 1) Remove non-mass measurements

In [3]:
mass_data = raw_data[raw_data.units=='mg/m^2 (dry weight)']

In [4]:
metadata = pd.read_csv('data/groups_per_reference.csv')
mass_data_with_meta = mass_data.merge(metadata,left_on='reference', right_on='Paper',how='left')

In [5]:
mass_data_with_meta.groupby('Standard groups')['site'].nunique()

Standard groups
Acari                  9
Arthropods           156
Collembola, Acari     47
Formicidae            49
Macroarthropods        4
Microarthropods        2
Orthoptera             1
Some arthropods        4
Name: site, dtype: int64

In [6]:
accepted_taxa = ['Acari','Arthropods','Collembola, Acari','Formicidae','Isoptera']
valid_data = mass_data_with_meta[mass_data_with_meta['Standard groups'].isin(accepted_taxa)]

In [7]:
valid_data.loc[valid_data['sub-class']=='Acari','aggregated taxon'] = 'Acari'
valid_data.loc[valid_data['sub-class']=='Collembola','aggregated taxon'] = 'Collembola'
valid_data.loc[valid_data['super-family']=='Isoptera','aggregated taxon'] = 'Isoptera'
valid_data.loc[valid_data['family']=='Formicidae','aggregated taxon'] = 'Formicidae'
valid_data.loc[valid_data['aggregated taxon'].isna(),'aggregated taxon'] = 'Other'

/home/yinonbaron/.local/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/yinonbaron/.local/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
soil_data = valid_data[valid_data['aggregated environment'] =='soil/litter']
canopy_data = valid_data[valid_data['aggregated environment'] =='plants']
surface_data = valid_data[valid_data['aggregated environment'] =='above ground']

In [9]:
soil_data.pivot_table(columns='aggregated taxon',values='site',index='aggregated biome', aggfunc='nunique')

aggregated taxon,Acari,Collembola,Formicidae,Isoptera,Other
aggregated biome,,,,,
Boreal Forests/Taiga,11.0,10.0,NaN,NaN,4.0
Deserts and Xeric Shrublands,NaN,NaN,4.0,NaN,NaN
Pasture,6.0,6.0,1.0,NaN,NaN
Shrubland/Grassland,NaN,NaN,14.0,NaN,NaN
Temperate Forests,38.0,37.0,21.0,NaN,23.0
"Temperate Grasslands, Savannas and Shrublands",14.0,16.0,9.0,3.0,12.0
Tropical and Subtropical Forests,5.0,6.0,15.0,7.0,7.0
"Tropical and Subtropical Grasslands, Savannas and Shrublands",2.0,2.0,2.0,5.0,3.0
Tundra,47.0,34.0,1.0,1.0,14.0


In [10]:
from uncertainties import unumpy as unp

In [11]:
soil_site_data = soil_data.groupby(['aggregated taxon','aggregated biome','site'])['numerical value'].sum()
soil_biome_means = soil_site_data.groupby(['aggregated taxon','aggregated biome']).agg(['mean','count','sem'])
soil_biome_means.reset_index().pivot_table(columns='aggregated taxon',index='aggregated biome').to_csv('results/biome_means.csv')

In [27]:
soil_biome_means_unp.pivot_table(index='aggregated taxon',columns='aggregated biome',values='numerical value',aggfunc=sum).to_csv('results/table1.csv')

In [12]:
soil_biome_means_unp = soil_site_data.groupby(['aggregated taxon','aggregated biome']).apply(lambda x: unp.uarray(nominal_values= x.mean(),std_devs=x.sem()))
soil_biome_means_unp = soil_biome_means_unp.reset_index()
biome_area = pd.read_csv('data/aggregated biomes data.csv')
biome_area1 = biome_area.groupby('aggregated biome 1')['area'].sum()
biome_area2 = biome_area.groupby('aggregated biome 2')['area'].sum()
# .pivot_table(columns='aggregated taxon',index='aggregated biome')
soil_biome_means_unp1 = soil_biome_means_unp.merge(pd.DataFrame(biome_area1),left_on='aggregated biome',right_index=True)
soil_biome_means_unp2 = soil_biome_means_unp[soil_biome_means_unp['aggregated taxon'].isin(['Formicidae','Isoptera'])]
soil_biome_means_unp2.loc[soil_biome_means_unp2.loc[:,'aggregated biome'].str.contains('Forest'),'aggregated biome'] = 'Forests'
soil_biome_means_unp2.loc[soil_biome_means_unp2.loc[:,'aggregated biome'].str.contains('Grass'),'aggregated biome'] = 'Shrubland/Grassland'
soil_biome_means_unp2 = soil_biome_means_unp2.merge(pd.DataFrame(biome_area2),left_on='aggregated biome',right_index=True)
part1 = soil_biome_means_unp1[soil_biome_means_unp1['aggregated taxon'].isin(['Acari','Collembola','Other'])]
part2 = soil_biome_means_unp2[soil_biome_means_unp2['aggregated taxon'].isin(['Formicidae','Isoptera'])]

/home/yinonbaron/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/home/yinonbaron/.local/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [13]:
part1.loc[:,'Total'] = (part1.loc[:,'numerical value']*part1.loc[:,'area']).values
part1.pivot_table(values='Total',index='aggregated taxon',columns='aggregated biome',aggfunc=sum)/1e18

aggregated biome,Boreal Forests/Taiga,Pasture,Temperate Forests,"Temperate Grasslands, Savannas and Shrublands",Tropical and Subtropical Forests,"Tropical and Subtropical Grasslands, Savannas and Shrublands",Tundra
aggregated taxon,,,,,,,
Acari,0.0111+/-0.0024,(6.4+/-2.6)e-05,0.0090+/-0.0020,0.0017+/-0.0005,0.021+/-0.020,0.0008+/-0.0005,0.00149+/-0.00029
Collembola,0.0060+/-0.0021,0.0+/-0,0.08+/-0.04,0.00101+/-0.00021,0.0014+/-0.0006,0.000128+/-0.000010,0.0030+/-0.0011
Other,0.0025+/-0.0009,NaN,0.0121+/-0.0027,0.006+/-0.005,0.0009+/-0.0004,0.00032+/-0.00014,0.00051+/-0.00027


In [14]:
part1.groupby('aggregated taxon').Total.sum()/1e18

aggregated taxon
Acari         0.046+/-0.020
Collembola      0.09+/-0.04
Other         0.023+/-0.006
Name: Total, dtype: object

In [15]:
part2 = part2[part2['aggregated biome']!='Tundra']
part2.loc[:,'Total'] = (part2.loc[:,'numerical value']*part2.loc[:,'area']).values
part2.pivot_table(values='Total',index='aggregated taxon',columns='aggregated biome',aggfunc=sum)/1e18

aggregated biome,Forests,Shrubland/Grassland
aggregated taxon,,
Formicidae,0.0050+/-0.0012,0.031+/-0.007
Isoptera,0.026+/-0.020,0.07+/-0.05


In [16]:
part2.groupby('aggregated taxon').Total.sum()/1e18

aggregated taxon
Formicidae    0.036+/-0.007
Isoptera        0.10+/-0.06
Name: Total, dtype: object

In [18]:
total_soil = (part1.Total.sum()+part2.Total.sum())/1e18
total_soil

0.2902956810128281+/-0.07171740599289464

In [19]:
canopy_biome_mean = canopy_data.groupby(['aggregated biome','site'])['numerical value'].sum().groupby('aggregated biome').agg(['mean','sem'])
canopy_biome_mean = canopy_biome_mean.merge(pd.DataFrame(biome_area1),left_index=True,right_index=True)
canopy_biome_mean['unp']=unp.uarray(nominal_values=canopy_biome_mean['mean'],std_devs=canopy_biome_mean['sem'])
canopy_biome_mean['Total'] = canopy_biome_mean.unp*canopy_biome_mean.area
canopy_biome_mean.Total/1e18

Boreal Forests/Taiga                0.0121+/-0.0018
Temperate Forests                   0.0027+/-0.0007
Tropical and Subtropical Forests      0.010+/-0.009
Name: Total, dtype: object

In [20]:
total_canopy = canopy_biome_mean.Total.sum()/1e18

In [21]:
surface_biome_mean = surface_data.groupby(['aggregated biome','site'])['numerical value'].sum().groupby('aggregated biome').agg(['mean','sem'])
surface_biome_mean = surface_biome_mean.merge(pd.DataFrame(biome_area1),left_index=True,right_index=True)
surface_biome_mean['unp']=unp.uarray(nominal_values=surface_biome_mean['mean'],std_devs=surface_biome_mean['sem'])
surface_biome_mean['Total'] = surface_biome_mean.unp*surface_biome_mean.area
surface_biome_mean.Total/1e18

Crops                                             0.0079+/-0.0016
Pasture                                           0.0004+/-0.0004
Temperate Grasslands, Savannas and Shrublands    0.00284532+/-nan
Name: Total, dtype: object

In [22]:
total_surface = surface_biome_mean.Total.sum()/1e18

In [23]:
total_surface+total_canopy+total_soil

0.32676189826555413+/-nan

In [24]:
print(canopy_data.site.nunique())
print(surface_data.site.nunique())

14
18
